In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics

In [7]:
tempIemisions = pd.read_csv("DataSetFinal.csv", index_col=0)


In [8]:
tempIemisions.describe()

,dt,Value,Country,Type,AverageTemperature,Mes
count,22356.000000,2.235600e+04,22356.000000,22356.000000,22356.000000,22356.000000
mean,2001.000000,6.270248e+04,14.000000,2.000000,9.230230,6.832797
std,6.633398,1.379788e+05,7.789055,0.816515,9.537685,3.484143
min,1990.000000,6.148900e+01,1.000000,1.000000,-34.665000,1.000000
25%,1995.000000,5.287999e+03,7.000000,1.000000,3.819500,4.000000
50%,2001.000000,1.352975e+04,14.000000,2.000000,10.129500,7.000000
75%,2007.000000,5.538075e+04,21.000000,3.000000,16.188250,10.000000
max,2012.000000,1.052238e+06,27.000000,3.000000,28.916000,13.000000


In [9]:
#fem X totes les columnes menys temp i y nomes temp el axis=1 es per indicar
#que es una label de les columnes
X = tempIemisions.drop('AverageTemperature', axis=1)
y = tempIemisions['AverageTemperature']

#dividim el dataSet en test 20% i entrenament 80%
#el random_state es la llavor que s'utilitzarà per randomitzar 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#creem l'algorisme del arbre de decisions i l'entrenem
regressor = DecisionTreeRegressor()
regressor.fit(X_train,y_train

#fem les prediccions
y_pred = regressor.predict(X_test)
df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})  
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred))) 

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')